# Real-world Data Wrangling

In this project, you will apply the skills you acquired in the course to gather and wrangle real-world data with two datasets of your choice.

You will retrieve and extract the data, assess the data programmatically and visually, accross elements of data quality and structure, and implement a cleaning strategy for the data. You will then store the updated data into your selected database/data store, combine the data, and answer a research question with the datasets.

Throughout the process, you are expected to:

1. Explain your decisions towards methods used for gathering, assessing, cleaning, storing, and answering the research question
2. Write code comments so your code is more readable

## 1. Gather data

In this section, you will extract data using two different data gathering methods and combine the data. Use at least two different types of data-gathering methods.

### **1.1.** Problem Statement

 *The City of Vancouver tracks not only every animal that comes into its shelters, but also those that are reported as lost by their owners. While the city does track those that are matched back to their owner, is it possible that an animal still tracked as lost has possibly been accounted for? If so, is it possible to find somewhat reliable means to match animals based on data entered into a report for lost ones?*

### **1.2.** Gather at least two datasets using two different data gathering methods

List of data gathering methods:

- Download data manually
- Programmatically downloading files [ X ]
- Gather data by accessing APIs [ X ]
- Gather and extract data from HTML files using BeautifulSoup
- Extract data from a SQL database

Each dataset must have at least two variables, and have greater than 500 data samples within each dataset.

For each dataset, briefly describe why you picked the dataset and the gathering method (2-3 full sentences), including the names and significance of the variables in the dataset. Show your work (e.g., if using an API to download the data, please include a snippet of your code). 

Load the dataset programmtically into this notebook.

#### City of Vancouver Animal Control Inventory - Lost and Found

This dataset is information from the City of Vancouver where an owner of an animal has reported them as lost. It also tracks those that were either reported as found or were matched by the shelter back to the owner. 

I chose this dataset because it will address what animals were reported as lost within the city. This does not cover every animal that was lost, however it does provide a large sample size for this metro area.

Further information on the dataset can be found [here](https://opendata.vancouver.ca/explore/dataset/animal-control-inventory-lost-and-found/information).

Type: JSON

Method: This data was gathered by querying the City of Vancouver's database with the standard Opendatasoft API. I am doing it this way because the data is updated daily, and this guarantees that the most up-to-date information will be used.

Dataset variables:

- *breed* - type of animal or breed that fits best.
- *color* - color of the animal's coat/fur.
- *date* - date that the animal was lost
- *name* - the given name of the animal being tracked (if known).
- *sex* - used to label the biological sex of the animal, as well as if they are spayed or neutered (marked with `F/S` or `M/N` accordingly). `X` = unknown
- *state* - the last state of being for the animal, i.e. `matched` or `lost`.

After some poking around, I found out that without a `group by` statement, the server only returns 100 results. By including a `group by` statement for all of the fields, this should theoretically drop duplicate values. I'm also filtering out anything before `1998-10-03`, as this is the earliest `DateImpounded` timestamp in the other dataset that will be used.

In [ ]:
import requests
import pandas as pd
import datetime

# "lf" is for *l*ost and *f*ound
lf_api_query = "https://opendata.vancouver.ca/api/explore/v2.1/catalog/datasets/animal-control-inventory-lost-and-found/records?where=date%20%3E%20%221998-10-02%22&group_by=date%2C%20breed%2C%20color%2C%20name%2C%20sex%2C%20state&order_by=date&limit=-1"
lf_data = requests.get(lf_api_query)
lf_data.raise_for_status()

In [2]:
lf_json = lf_data.json()

print(lf_json.keys())
print(lf_json['total_count'])

lf_json['results'][0:3]
print(type(lf_json['results']))     # all elements
print(type(lf_json['results'][0]))  # individual element

dict_keys(['total_count', 'results'])
17837
<class 'list'>
<class 'dict'>


Based on our mild digging above, we would want to load specifically the data in the `results` key as a Pandas DataFrame, since `results` is simply a `list` of `dict`'s, which Pandas.DataFrame's constructor can handle.

In [3]:
lf_df = pd.DataFrame(lf_json['results'])

#### City of Vancouver Animal Control Inventory - Register

This dataset is a "general record of each animal that has come into the custody" the City of Vancouver's animal control service.

I chose this dataset to have a record to compare all of the lost and found animals to in the event an animal is reported as lost and the City of Vancouver happens to have them, or someone very much like them, already processed into their database.

Like with the lost and found dataset, this data is updated daily. Because I must choose a different method to pull this data, I will download it programatically, as well as in CSV format just to make sure I cover all bases for this project.

Type: Semicolon (`;`) delimited "CSV" file.

Method: Programatic download via HTTP GET request

Dataset variables:

- *AnimalID* - Unique sequential number given to each entry.
- *Breed* - Type of animal.
- *ShotsDate* - Date when vaccinated.
- *Sex* - M = Male, F = Female, M/N = Male Neutered, F/S = Female Spayed.
- *ReceiptNumber* - Point of sales system of record receipt number.
- *DateImpounded* - Date first in custody of the City of Vancouver.
- *PitNumber* - Number identifying animal kennel, does not change while in custody of the city.
- *Name* - Name if known.
- *KennelNumber* - Kennel number displayed at the top of each kennel.
- *DispositionDate* - Date when animal was no longer under the control of the city.
- *Color* - Color of coat.
- *Code* - Walk-ability index (*Green = easy, Yellow = moderate, Blue = hard*).
- *ApproxWeight* - Approximate weight of animal.
- *Age category* - Rough estimate of age - puppy, young adult, adult, senior.
- *Source* - Where the animal came from (Brought-in, Holding stray, Transferred).
- *Status* - Current state/disposition of animal.
- *ACO* - Animal control officer number or initials of employee.

In [4]:
# "reg" is for registry
reg_url = "https://opendata.vancouver.ca/api/explore/v2.1/catalog/datasets/animal-control-inventory-register/exports/csv?lang=en&timezone=America%2FChicago&use_labels=true&delimiter=%3B"
reg_data = requests.get(reg_url)
reg_data.raise_for_status()

In [ ]:
# Save contents to a file labeled with today's date.
file_dl_date = datetime.date.today()
filename = f"vancouver-ac-registry_{file_dl_date.strftime('%Y%m%d')}.csv"
relativepath = "./datasets/" + filename

# Write dataset JSON stored as binary to target file
with open(relativepath, mode="wb") as f:
    f.write(reg_data.content)

We'll make the `AnimalID` column our index column for our DataFrame since it's essentially a built-in order for the animals. Following this, we'll start assessing our data.

In [6]:
reg_df = pd.read_csv(relativepath, sep=";", index_col='AnimalID')

## 2. Assess data

When we are assessing data, we are on the lookout for **quality** and **tidiness** (structural) issues.

**Quality Issues:**
- Completeness - The collected data is sufficient for addressing specific problems.
- Validity - Data conforms to the defined schema.
- Accuracy - Data accurately represents the reality it is describing.
- Consistency - A standard format is followed. Data matches that which can be found in other sources.
- Uniqueness - non-duplicate or overlapping values in the data.

**Tidiness Issues:**
- Each variable forms an individual column, i.e. color, name, birth year, etc. 
  - This also means that each column only contains one variable, or "factor that varies."
- Each observation forms an individual row, i.e. red, Scott, 1997, etc.
  - As with columns, only a single observation per row.
- Each type of observational unit forms a table, i.e. a table of immediate family members.

### Quality Issue 1:

We'll use the `.head()` and `.tail()` methods to quickly browse the dataset for anything that sticks out. This helps us ensure completeness to some degree, as well as validity and consistency.

In [ ]:
# Inspecting the dataframe visually
lf_df.head()

,date,breed,color,name,sex,state
0,1999-01-03T00:00:00+00:00,Rotty X Shep,Black & Tan,Tex,M/N,Lost
1,1999-01-04T00:00:00+00:00,Dog,Light Colour,None,M/N,Found
2,1999-01-04T00:00:00+00:00,Golden Lab X,Black & Tan,Oscar,M,Lost
3,1999-01-04T00:00:00+00:00,Shep X,Black & Tan,None,F,Found
4,1999-01-04T00:00:00+00:00,Shep X Collie,Black & Tan,Angel,F,Lost


In [8]:
lf_df.tail()

,date,breed,color,name,sex,state
17832,2025-07-19T00:00:00+00:00,Cat- DSH,Grey,Sissy,F/S,Lost
17833,2025-07-20T00:00:00+00:00,Cat - DSH - British Shorthair,Grey and white,Sleepy,F/S,Lost
17834,2025-07-20T00:00:00+00:00,Cat - DSH - Devon Rex,Grey w/brown,Trixie,F/S,Lost
17835,2025-07-21T00:00:00+00:00,Cat - DSH,Creamy white/brown,Hylia,F/S,Lost
17836,2025-07-22T00:00:00+00:00,.Unknown Breed Mix,Cream,Bebe,F/S,Lost


In [78]:
lf_df.sample(5)  # random sampling to see if anything else jumps out

,date,breed,color,name,sex,state
16282,2022-10-31T00:00:00+00:00,Golden Retriever,Cream,Kyra,F,Matched
4696,2003-05-22T00:00:00+00:00,Akita,Grey,None,F,Matched
15383,2021-08-21T00:00:00+00:00,Pit Bull,Black,Not Provided,X,Lost
12013,2016-05-19T00:00:00+00:00,Cat - DLH x 2,Blk/White & Siamese,Trixie (F) & Mike (M),F,Matched
1120,2000-01-29T00:00:00+00:00,German Shepherd,Brown & Black,Shiva,F,Lost


Thankfully, there are no duplicate entries in this data (as of 20250723), indicating each row has "uniqueness."

In [9]:
lf_df.duplicated().value_counts()

False    17837
Name: count, dtype: int64

There are a lot of colors in the `color` column, which indicates potential a lack of consistency and validity.

In [56]:
print("There are", lf_df['color'].unique().shape[0], "unique strings in the color column.")

There are 3399 unique strings in the color column.


One prime example is this hamster, which has `Golden/Blonde` fur. This indicates there is not a standardized process for determining and labeling fur color.

In [71]:
lf_df.iloc[16633]

date     2023-06-26T00:00:00+00:00
breed                      Hamster
color                Golden/Blonde
name                        Chivis
sex                              M
state                         Lost
Name: 16633, dtype: object

I am also curious about how many of those colors have consistency issues, for example using an ampersand instead of an "and". The use of slashes may also indicate `color` labeling similar to `Chivis`, as seen immediately above.

In [88]:
num_colors_amp = pd.Series(lf_df.color.unique()).str.contains(r'&', na=False).value_counts().iloc[1]
num_colors_and = pd.Series(lf_df.color.unique()).str.contains(r' and ', na=False).value_counts().iloc[1]
num_colors_fwdsl = pd.Series(lf_df.color.unique()).str.contains(r'/', na=False).value_counts().iloc[1]

print(num_colors_amp, "unique color strings contain '&' in the color description while", num_colors_and, "contain 'and'.")
print(num_colors_fwdsl, " unique colors contain a forward slash (/).")

1151 unique color strings contain '&' in the color description while 191 contain 'and'.
1341  unique colors contain a forward slash (/).


Issue and justification: Aside from lacking completeness, a lot of the columns do not have a consistent format:
Currently, "&" is primarily used in place of "and" in the `color` column. This matters when trying to answer our question as this means that matching based on `color` will different approaches (fuzzy matching, tokenization, etc.). 

As a side note, the formatting in the `breed` column is also quite inconsistent, but this is addressed further in the tidiness section.

### Quality Issue 2:

We'll do a head, tail, and sample again.

In [13]:
reg_df.head()

,Breed,ShotsDate,Sex,ReceiptNumber,DateImpounded,PitNumber,Name,KennelNumber,DispositionDate,Color,Code,ApproxWeight,Age category,Source,Status,ACO
AnimalID,,,,,,,,,,,,,,,,
28066,"Rat, Fancy x 1 Male",NaN,M,n/c,2017-04-13,NaN,Obi-Wan,200,2018-01-04,White/Brown,NaN,NaN,Adult,HOLDING STRAY,Sold,40
28070,Doberman Pinscher,NaN,M,DI 17-213469,2017-04-14,NaN,Loki,200,2017-04-15,Black,NaN,NaN,Young Adult,HOLDING STRAY,Redeemed,32
28072,Chihuahua,NaN,M,DI 17-213705,2017-04-15,NaN,Hugo,200,2017-04-16,Black,NaN,NaN,Young Adult,HOLDING STRAY,Redeemed,42
28074,Chihuahua,NaN,F,DI 17-214366 DB,2017-04-16,NaN,ChiChi,200,2017-04-18,Cream,NaN,NaN,Adult,HOLDING STRAY,Redeemed,12/26
28077,Chihuahua,NaN,F,NaN,2017-04-16,NaN,Oreo,200,2017-04-18,Black,NaN,NaN,Senior,POLICE-EMERGENCY,Redeemed,NaN


In [101]:
reg_df.tail()

,Breed,ShotsDate,Sex,ReceiptNumber,DateImpounded,PitNumber,Name,KennelNumber,DispositionDate,Color,Code,ApproxWeight,Age category,Source,Status,ACO
AnimalID,,,,,,,,,,,,,,,,
27972,Terrier X,2017-03-09,M/N,DA 17-199342 KW,2017-03-05,NaN,Wallaby (new name),200,2017-03-14,Brown,Yellow,NaN,Young Adult,HOLDING STRAY,Sold,42
27973,"Guinea Pig, baby",NaN,M,DA 17-196342,2017-03-05,NaN,NaN,200,2017-03-10,Red,NaN,NaN,Puppy,TRANSFERRED,Sold,rb
27977,Germ. Shepherd,NaN,M/N,DI 17-196766 -MA,2017-03-07,NaN,Murphy,200,2017-03-07,Black,NaN,NaN,Adult,COMPLAINT,Redeemed,12
27983,Maltese X,NaN,M/N,NaN,2017-03-08,NaN,Yogui,200,2017-03-09,Cream w/black/tan,NaN,NaN,Adult,BROUGHT-IN,Redeemed,NaN
27984,Husky,NaN,M,DI 17-198335 -EL,2017-03-08,NaN,Hei Hei,200,2017-03-12,Black/White,NaN,NaN,Adult,HOLDING STRAY,Redeemed,40


In [100]:
reg_df.sample(5)

,Breed,ShotsDate,Sex,ReceiptNumber,DateImpounded,PitNumber,Name,KennelNumber,DispositionDate,Color,Code,ApproxWeight,Age category,Source,Status,ACO
AnimalID,,,,,,,,,,,,,,,,
20829,Mini Aussie Shepherd,NaN,F,10-193435DI - TU,2010-06-16,20082.0,Lacey,200,2010-06-17,Tri Colour,NaN,30 lbs.,NaN,COMPLAINT,Redeemed,20
1538,Rottweiler,2000-01-19,M/N,"9886,BE",2000-01-04,20089.0,Jonah,200,NaN,Black & Tan,NaN,NaN,NaN,NaN,Sold,5
33787,Labradoodle,NaN,F,RHF,2022-09-08,NaN,Lucky,400,2022-09-08,Black&White,NaN,NaN,Adult,BROUGHT-IN,Ride Home Free,NaN
4616,Min Pin X,NaN,F/S,12247 - #4,2001-07-24,20063.0,Hula,200,NaN,Tan,NaN,NaN,NaN,HOLDING STRAY,Redeemed,4
12693,Chow X,NaN,M/N,16900 - skj,2005-08-24,20085.0,Nikita,200,NaN,Gold with Black,NaN,110 lbs.,NaN,HOLDING STRAY,Redeemed,14


Since there are duplicates, we will need to drop those.

In [14]:
reg_df.duplicated().value_counts()

False    26038
True        41
Name: count, dtype: int64

Using the results from the `.info()` method, we can quickly visually parse about how many null values there are per column.

In [15]:
reg_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 26079 entries, 28066 to 27984
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Breed            26071 non-null  object 
 1   ShotsDate        3752 non-null   object 
 2   Sex              25793 non-null  object 
 3   ReceiptNumber    21680 non-null  object 
 4   DateImpounded    26079 non-null  object 
 5   PitNumber        17763 non-null  float64
 6   Name             23330 non-null  object 
 7   KennelNumber     26058 non-null  object 
 8   DispositionDate  12313 non-null  object 
 9   Color            26034 non-null  object 
 10  Code             1173 non-null   object 
 11  ApproxWeight     11643 non-null  object 
 12  Age category     8028 non-null   object 
 13  Source           24105 non-null  object 
 14  Status           26070 non-null  object 
 15  ACO              21311 non-null  object 
dtypes: float64(1), object(15)
memory usage: 3.4+ MB


`Age category` is the only column with a space character, as well as not being in `CamelCase`. Additionally, these columns are `CamelCase` instead of all lowercase, like the column labels in the `lost and found` dataset.

In [16]:
reg_df['Age category'].value_counts()   # Puppy being a category assumes the animal is a dog

Age category
Adult          4909
Young Adult    1543
Senior         1132
Puppy           444
Name: count, dtype: int64

Issue and justification: Immediately, we can see that there are a lot of null values, especially for `Code`, `ShotsDate`, and `Age category` columns. Each of those have no more than about 30% of the values filled. It would be difficult, however, to infer anything to answer our question using any of these columns. 

The best columns would probably be `name`, `color`, `sex`, and `breed` for making inferences, so this means we can probably just drop the columns that are mostly null. For `name` and the other column.s previousy mentioned, we can probably just replace the null values with something like "Unknown"

All of these null values make correlating animals together based on similar factors less reliable, especially if something critical like the name isn't disclosed or the color is not reported as the staff intaking animals would describe. Names are also likely the highest differentiator between two animals with similar breeds and coat colors, as well.

There are also duplicate values, but those will be easy to handle. The biggest issue I feel is the insane abundance of missing values.

### Tidiness Issue 1:

I'm of the opinion that validating the 3 qualities of tidiness is most easily began visually with `.head()/.tail()`, then drilling down programatically with `.value_counts()`, queries, `.columns`, and more. 

In [ ]:
lf_df.head()  # lets ground overselves in the data again

,date,breed,color,name,sex,state
0,1999-01-03T00:00:00+00:00,Rotty X Shep,Black & Tan,Tex,M/N,Lost
1,1999-01-04T00:00:00+00:00,Dog,Light Colour,None,M/N,Found
2,1999-01-04T00:00:00+00:00,Golden Lab X,Black & Tan,Oscar,M,Lost
3,1999-01-04T00:00:00+00:00,Shep X,Black & Tan,None,F,Found
4,1999-01-04T00:00:00+00:00,Shep X Collie,Black & Tan,Angel,F,Lost


In [103]:
lf_df.tail()

,date,breed,color,name,sex,state
17832,2025-07-19T00:00:00+00:00,Cat- DSH,Grey,Sissy,F/S,Lost
17833,2025-07-20T00:00:00+00:00,Cat - DSH - British Shorthair,Grey and white,Sleepy,F/S,Lost
17834,2025-07-20T00:00:00+00:00,Cat - DSH - Devon Rex,Grey w/brown,Trixie,F/S,Lost
17835,2025-07-21T00:00:00+00:00,Cat - DSH,Creamy white/brown,Hylia,F/S,Lost
17836,2025-07-22T00:00:00+00:00,.Unknown Breed Mix,Cream,Bebe,F/S,Lost


Based on the output above, the columns are each a distinct type of variable. It does seem, however, some columns store multiple variables in a single datapoint, especially `sex`, which indicates the reproductive sterility of an animal. This column, and another to be investigated further below, can be split into additional columns.

In [104]:
lf_df.sex.value_counts()

sex
M      5099
M/N    4551
F      3899
F/S    3681
X       216
Name: count, dtype: int64

In [19]:
print("There are", lf_df.breed.unique().shape[0], "unique breeds in the dataset.")

breed_s = pd.Series(lf_df.breed.unique()).sort_values(na_position='first', ignore_index=True)
breed_s.head(10)

There are 4041 unique breeds in the dataset.


0                            None
1          (Miniature) Pomeranian
2              .Unknown Breed Mix
3      1 Pit Bull & 1 Terrier mix
4    1 Pitbull & 1 Bernese/Poodle
5             1/2 Pit & 1/2 Presa
6                      2 Dachunds
7                2 Great Pyrenese
8                       2 Husky X
9                 2 Labs & 1 Shep
dtype: object

In [20]:
breed_s.tail(10)

4031    stratfordshire terrier
4032                     tabby
4033    tabby Grey black brown
4034       tabby short haireds
4035               terrier (?)
4036                 terrier X
4037             terrier X pug
4038      very small ( 5 lbs )
4039             westy terrier
4040              yorkie cross
dtype: object

In [21]:
breed_s.str.extractall("(.*\d.*)")

,,0
,match,
3,0,1 Pit Bull & 1 Terrier mix
4,0,1 Pitbull & 1 Bernese/Poodle
5,0,1/2 Pit & 1/2 Presa
6,0,2 Dachunds
7,0,2 Great Pyrenese
...,...,...
3640,0,Staff Terrier #2
3662,0,Stafforshire Terrier 14 months
3973,0,lab and 2nd dog German Sheper


Issue and justification: It seems the `breed` column could be useful for finding single rows with multiple animals reported (e.g., `1 Pit Bull & 1 Terrier mix`). A more tidy way to do this would be splitting those out into separate rows. If the report needs to be associated with those from the same report, there can be a report ID that all applicable reports share, however I will not need that information to answer my question.

By having multiple animals in one row, this not only inaccurately represents the number of animals reported missing, but also makes grouping or filtering by the reported breed unreliable.

The `sex` column could also be split into two columns, where one solely stores the biological sex of the animal and the other stores if they are neutered or spayed. This can just be a boolean value for simplicity's sake.

### Tidiness Issue 2: 

In [102]:
reg_df.head(10)

,Breed,ShotsDate,Sex,ReceiptNumber,DateImpounded,PitNumber,Name,KennelNumber,DispositionDate,Color,Code,ApproxWeight,Age category,Source,Status,ACO
AnimalID,,,,,,,,,,,,,,,,
28066,"Rat, Fancy x 1 Male",NaN,M,n/c,2017-04-13,NaN,Obi-Wan,200,2018-01-04,White/Brown,NaN,NaN,Adult,HOLDING STRAY,Sold,40
28070,Doberman Pinscher,NaN,M,DI 17-213469,2017-04-14,NaN,Loki,200,2017-04-15,Black,NaN,NaN,Young Adult,HOLDING STRAY,Redeemed,32
28072,Chihuahua,NaN,M,DI 17-213705,2017-04-15,NaN,Hugo,200,2017-04-16,Black,NaN,NaN,Young Adult,HOLDING STRAY,Redeemed,42
28074,Chihuahua,NaN,F,DI 17-214366 DB,2017-04-16,NaN,ChiChi,200,2017-04-18,Cream,NaN,NaN,Adult,HOLDING STRAY,Redeemed,12/26
28077,Chihuahua,NaN,F,NaN,2017-04-16,NaN,Oreo,200,2017-04-18,Black,NaN,NaN,Senior,POLICE-EMERGENCY,Redeemed,NaN
28079,Budgie,NaN,F,NaN,2017-04-18,NaN,NaN,100,2017-04-19,Green/Yellow/Black,NaN,NaN,Young Adult,BROUGHT-IN,Passed Away,NaN
28083,Guinea Pig,NaN,F,NaN,2017-04-18,NaN,Penny,200,2017-06-07,Tri colored - Large,NaN,NaN,Young Adult,COMPLAINT,Sold,NaN
28086,Lab X,NaN,M/N,NaN,2017-04-18,NaN,Farley,400,2017-04-19,Yellow,NaN,NaN,Senior,HOLDING STRAY,Ride Home Free,35
28093,Shih Tzu,NaN,M/N,DI 17-216041,2017-04-23,NaN,TT,200,2017-04-23,Cream,NaN,NaN,Senior,HOLDING STRAY,Redeemed,26/34


In [106]:
reg_df.sort_index().head()

,Breed,ShotsDate,Sex,ReceiptNumber,DateImpounded,PitNumber,Name,KennelNumber,DispositionDate,Color,Code,ApproxWeight,Age category,Source,Status,ACO
AnimalID,,,,,,,,,,,,,,,,
1,Pit Bull,2005-06-18,M/N,17057 ks,2005-06-12,20038.0,Taz,200,NaN,Tan,NaN,45,NaN,HOLDING STRAY,Sold,3
2,English Setter,NaN,M/N,20372 BC3,2006-03-19,20041.0,Dudley,200,NaN,White & Brown,NaN,100lbs,NaN,HOLDING STRAY,Redeemed,NaN
3,Lab X,NaN,M/N,N/C BC,2006-07-24,40041.0,Evander,400,NaN,Black,NaN,NaN,NaN,HOLDING STRAY,Ride Home Free,5
4,Pomeranian,NaN,M,"22171,MC",2006-09-08,20017.0,Chewbaca,200,NaN,Brown,NaN,10 lbs.,NaN,HOLDING STRAY,Redeemed,14
5,American Bulldog X,2006-10-07,M/N,22565 - skj,2006-09-06,20053.0,Cadillac,200,NaN,White with Brown Patch on Eye,NaN,80 lbs,NaN,HOLDING STRAY,Sold,22


Skimming through the breed column revealed multiple occurrences of a string like "`<breed-name> X`". Immediately, I am unsure what this may indicate, but I have a presumption that I will elaborate upon shortly.

In [125]:
reg_df.query('AnimalID == 3').iloc[0]  # iloc will return a series, which is a bit easier to read for a single row

Breed                       Lab X
ShotsDate                     NaN
Sex                           M/N
ReceiptNumber              N/C BC
DateImpounded          2006-07-24
PitNumber                 40041.0
Name                      Evander
KennelNumber                  400
DispositionDate               NaN
Color                       Black
Code                          NaN
ApproxWeight                  NaN
Age category                  NaN
Source              HOLDING STRAY
Status             Ride Home Free
ACO                             5
Name: 3, dtype: object

Looking a little further, it looks like `Lab X` is incredibly common.

In [25]:
reg_df['Breed'].str.extractall("(.*[lL]ab.*)").value_counts()

0                     
Lab X                     872
Lab                       571
Labrador                  495
Labrador X                 78
Shep X Lab                 78
                         ... 
Black Lab X Pit             1
Black Lab/GSD               1
Black X Lab                 1
Heeler X Lab X Bassett      1
rotty X lab                 1
Name: count, Length: 490, dtype: int64

Issue and justification: There are a few places where some improvements can be made. 
- `Name` - Some rows contain "`(New name)`," which is tracking two factors of data in a single column.
  - This could be resolved with another column of boolean values, but I do not think it will be entirely necessary for our cause. Therefore, we will remove occurrences of `(New name)`
- `Breed` - Some breeds have "`mix`" while others have "`X`". Presumably, `X` is also for "mix," but this is merely an assumption. Additionally, `Black Lab` is not a breed, the breed is Labrador Retriever. 
  - One way to handle this is to see if the `Breed` field contains the `Color` of the animal, then trimming that out if it does.

Another general issue is that this field has inconsistent capitalization. Eventually, I will normalize all of the string fields so that they are all in lowercase.

## 3. Clean data
Clean the data to solve the 4 issues corresponding to data quality and tidiness found in the assessing step. **Make sure you include justifications for your cleaning decisions.**

After the cleaning for each issue, please use **either** the visually or programatical method to validate the cleaning was succesful.

At this stage, you are also expected to remove variables that are unnecessary for your analysis and combine your datasets. Depending on your datasets, you may choose to perform variable combination and elimination before or after the cleaning stage. Your dataset must have **at least** 4 variables after combining the data.

In [26]:
# Make copies of the datasets to ensure the raw dataframes are not impacted
lf_df_clean = lf_df.copy()
reg_df_clean = reg_df.copy()

### **Quality Issue 1: FILL IN**

For the issue of ______ I will

In [27]:
# FILL IN - Apply the cleaning strategy.


In [28]:
# FILL IN - Validate the cleaning was successful

Justification: *FILL IN*

### **Quality Issue 2: FILL IN**

In [29]:
#FILL IN - Apply the cleaning strategy

In [30]:
#FILL IN - Validate the cleaning was successful

Justification: *FILL IN*

### **Tidiness Issue 1: FILL IN**

In [31]:
#FILL IN - Apply the cleaning strategy

In [32]:
#FILL IN - Validate the cleaning was successful

Justification: *FILL IN*

### **Tidiness Issue 2: FILL IN**

In [33]:
#FILL IN - Apply the cleaning strategy

In [34]:
#FILL IN - Validate the cleaning was successful

Justification: *FILL IN*

### **Remove unnecessary variables and combine datasets**

Depending on the datasets, you can also peform the combination before the cleaning steps.

In [35]:
#FILL IN - Remove unnecessary variables and combine datasets

## 4. Update your data store
Update your local database/data store with the cleaned data, following best practices for storing your cleaned data:

- Must maintain different instances / versions of data (raw and cleaned data)
- Must name the dataset files informatively
- Ensure both the raw and cleaned data is saved to your database/data store

In [36]:
#FILL IN - saving data

## 5. Answer the research question

### **5.1:** Define and answer the research question 
Going back to the problem statement in step 1, use the cleaned data to answer the question you raised. Produce **at least** two visualizations using the cleaned data and explain how they help you answer the question.

*Research question:* FILL IN from answer to Step 1

In [37]:
#Visual 1 - FILL IN

*Answer to research question:* FILL IN